In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import sys
import os
import logging
import galsim
import random
import cmath as cm
import math
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply, Reshape, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential
import scipy
import scipy.stats as stats
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo
%matplotlib inline
from tensorflow.keras.layers import Conv2D, Input, Dense, Dropout, MaxPool2D, Flatten,  Reshape, UpSampling2D, Cropping2D, Conv2DTranspose, PReLU, Concatenate, Lambda, BatchNormalization, concatenate
import tensorflow as tf
import seaborn as sns

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import sys
import os
import logging
import galsim
import random
import cmath as cm
import math
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply, Reshape, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, Input, Dense, Dropout, MaxPool2D, Flatten,  Reshape, UpSampling2D, Cropping2D, Conv2DTranspose, PReLU, Concatenate, Lambda, BatchNormalization, concatenate, LeakyReLU

In [3]:
nb_of_bands = 1

In [4]:
#### Create encooder
def build_encoder(latent_dim, hidden_dim, filters, kernels,nb_of_bands, conv_activation='softplus', dense_activation='softplus'):
    """
    Return encoder as model
    latent_dim : dimension of the latent variable
    hidden_dim : dimension of the dense hidden layer
    filters: list of the sizes of the filters used for this model
    list of the size of the kernels used for each filter of this model
    conv_activation: type of activation layer used after the convolutional layers
    dense_activation: type of activation layer used after the dense layers
    nb_of bands : nb of band-pass filters needed in the model
    """
    input_layer = Input(shape=(64,64,nb_of_bands))

    r = Reshape((64,64,nb_of_bands))(input_layer)
    h = BatchNormalization()(r)
    h_shortcut = h
    for i in range(len(filters)):
        h = Conv2D(filters[i], (kernels[i],kernels[i]), activation=conv_activation, padding='same')(h)
        h = PReLU()(h)
        h = Conv2D(filters[i], (kernels[i],kernels[i]), activation=conv_activation, padding='same', strides=(2,2))(h)
        h = PReLU()(h)
    h = Flatten()(h)
    h = Dense(hidden_dim, activation=dense_activation)(h)
    h = PReLU()(h)
    mu = Dense(latent_dim)(h)
    sigma = Dense(latent_dim, activation='softplus')(h)
    return Model(input_layer, [mu, sigma, h_shortcut])

In [5]:
batch_size = 100 

input_shape = (64,64,nb_of_bands)
hidden_dim = 256
filters = [32,64, 128, 256]
kernels = [3,3,3,3]
nb_of_bands = 1
latent_dim = 32

# Build the encoder
encoder = build_encoder(latent_dim, hidden_dim, filters, kernels, nb_of_bands)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 64, 64, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 64, 64, 1)    4           reshape[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   320         batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
p_re_lu (P

In [7]:
#### Create encooder
def build_decoder(input_shape, latent_dim, hidden_dim, filters, kernels, conv_activation='softplus', dense_activation='softplus'):
    """
    Return decoder as model
    input_shape: shape of the input data
    latent_dim : dimension of the latent variable
    hidden_dim : dimension of the dense hidden layer
    filters: list of the sizes of the filters used for this model
    list of the size of the kernels used for each filter of this model
    conv_activation: type of activation layer used after the convolutional layers
    dense_activation: type of activation layer used after the dense layers
    """
    input_layer = Input(shape=(latent_dim,))
    h_shortcut = Input(shape=(64,64,1))
    h = Dense(hidden_dim, activation=dense_activation)(input_layer)
    h = PReLU()(h)
    w = int(np.ceil(input_shape[0]/2**(len(filters))))
    h = Dense(w*w*filters[-1], activation=dense_activation)(h)
    h = PReLU()(h)
    h = Reshape((w,w,filters[-1]))(h)
    for i in range(len(filters)-1,-1,-1):
        h = Conv2DTranspose(filters[i], (kernels[i],kernels[i]), activation=conv_activation, padding='same', strides=(2,2))(h)
        h = PReLU()(h)
        h = Conv2DTranspose(filters[i], (kernels[i],kernels[i]), activation=conv_activation, padding='same')(h)
        h = PReLU()(h)
    h = Add()([h, h_shortcut])
    h = Conv2D(input_shape[-1], (3,3), activation='sigmoid', padding='same')(h)
    cropping = int(h.get_shape()[1]-input_shape[0])
    if cropping>0:
        print('in cropping')
        if cropping % 2 == 0:
            h = Cropping2D(cropping/2)(h)
        else:
            h = Cropping2D(((cropping//2,cropping//2+1),(cropping//2,cropping//2+1)))(h)

    return Model([input_layer, h_shortcut], h)

In [8]:
# Build the decoder
decoder = build_decoder(input_shape, latent_dim, hidden_dim, filters, kernels, conv_activation=None, dense_activation=None)

In [9]:
decoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 256)          8448        input_2[0][0]                    
__________________________________________________________________________________________________
p_re_lu_9 (PReLU)               (None, 256)          256         dense_3[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 4096)         1052672     p_re_lu_9[0][0]                  
__________________________________________________________________________________________________
p_re_lu_10

In [10]:
import layers


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [19]:
def build_vanilla_vae(encoder, decoder, coeff_KL,full_cov=False):
    """
    Returns the model to train and parameters to plot relevant information during training using the VAEHistory callback
    """
    input_vae = Input(shape=encoder.input.shape[1:])
    mu, sigma, shortcut = encoder(input_vae)

    z, Dkl = layers.SampleMultivariateGaussian(full_cov=full_cov, add_KL=False, return_KL=True, coeff_KL=coeff_KL)([mu,sigma])
    
    vae = Model(input_vae, decoder([z,shortcut]))
    vae_utils = Model(input_vae, [*encoder(input_vae), z, Dkl, decoder([z,shortcut])])

    return vae, vae_utils, Dkl

In [21]:
vae, vae_utils, Dkl = build_vanilla_vae(encoder, decoder, coeff_KL = None)

In [22]:
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
model (Model)                   [(None, 32), (None,  2544420     input_6[0][0]                    
__________________________________________________________________________________________________
sample_multivariate_gaussian_2  [(None, 32), (None,) 0           model[4][0]                      
                                                                 model[4][1]                      
__________________________________________________________________________________________________
model_1 (Model)                 (None, 64, 64, 1)    3318497     sample_multivariate_gaussian_2[0]
          

In [14]:
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [18]:
plot_model(vae, to_file='test_resnet.png', show_shapes=True, show_layer_names=True)